In [ ]:
#Mounting to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install GitPython
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

In [ ]:
import git
import subprocess
import shutil
import os
import sys

class Retinanet:

  def __init__(self,working_dir=None):
    self.cwd=working_dir if working_dir != None else "/content"
  
  @staticmethod
  def import_model(cwd):

    #url="https://github.com/roboflow-ai/Monk_Object_Detection"
    url="https://github.com/karen-gishyan/Computer-vision-and-Drones-Thesis"
    repo_name=os.path.basename(url)
    path=os.path.join(cwd,repo_name) 
    print(path)
    
    if  not os.path.exists(path):
      git.Git(cwd).clone(url)
    
    else:    
      user_input=input("Non_Empty directory, replace? press y for Yes, n for No:")
      if user_input=="y":
        shutil.rmtree(path)
        git.Git(cwd).clone(url)
 
    os.chdir("/content/Computer-vision-and-Drones-Thesis/models/Modified Monk Retinanet/installation")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements_colab.txt"])
    os.chdir("/content")
    print("Requrements installed.")
  
  
cwd="/content/"
Retinanet().import_model(cwd)

In [ ]:
#! cd Monk_Object_Detection/3_mxrcnn/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install

In [ ]:
import os
import shutil


if not os.path.exists("/content/Root_Dir"): os.makedirs("/content/Root_Dir")

if not os.path.exists("/content/Coco_style") :os.makedirs("/content/Coco_style")
if not os.path.exists("/content/Coco_style/images") :os.makedirs("/content/Coco_style/images")
if not os.path.exists("/content/Coco_style/annotations") :os.makedirs("/content/Coco_style/annotations")

shutil.move("/content/Coco_style","/content/Root_Dir")

In [6]:
import shutil
os.chdir("/content/drive/My Drive/Bath thesis/") # takes some time.

train_img_path="Final-Data/Augmentation1/Augmentation 1/train_images"
train_annot_path="Final-Data/Augmentation1/Augmentation 1/train_annotations_json/instances_Train.json"
valid_img_path="Final-Data/1000 images without augmentation/valid_images"
valid_annot_path="Final-Data/1000 images without augmentation/instances_Val.json"
classes_path="Final-Data/1000 images without augmentation/classes alphabetical.txt"

def copy_to_root(train_img_path=train_img_path,valid_img_ath=valid_img_path,train_annot_path=train_annot_path,valid_annot_path=valid_annot_path,classes_path=classes_path):
  shutil.copytree(train_img_path,"/content/Root_Dir/Coco_style/images/Train")
  shutil.copytree(valid_img_path,"/content/Root_Dir/Coco_style/images/Val")
  shutil.copy(train_annot_path,"/content/Root_Dir/Coco_style/annotations")
  shutil.copy(valid_annot_path,"/content/Root_Dir/Coco_style/annotations")
  shutil.copy(classes_path,"/content/Root_Dir/Coco_style/annotations")
  os.rename("/content/Root_Dir/Coco_style/annotations/classes alphabetical.txt","/content/Root_Dir/Coco_style/annotations/classes.txt")

copy_to_root()

In [9]:
train_num_img=sum([1 for i in os.listdir("/content/Root_Dir/Coco_style/images/Train")])
valid_num_img=sum([1 for i in os.listdir("/content/Root_Dir/Coco_style/images/Val")])
print("There are {} train images and {} valid images.".format(train_num_img,valid_num_img))

There are 1400 train images and 200 valid images.


In [ ]:
#!pip install efficientnet_pytorch
#!pip install tensorboardx

#https://github.com/Tessellate-Imaging/Monk_Object_Detection/blob/master/example_notebooks/5_pytorch_retinanet/Train%20Resnet18%20-%20With%20validation%20Dataset.ipynb

import os
os.chdir("/content/Computer-vision-and-Drones-Thesis/models/Modified Monk Retinanet/lib")
from train_detector import Detector
model = Detector();

os.chdir("/content/")

troot_dir = "Root_Dir";
tcoco_dir = "Coco_style";
timg_dir = "images";
tset_dir = "Train";



vroot_dir = "Root_Dir";
vcoco_dir = "Coco_style";
vimg_dir = "images";
vset_dir = "Val";

model.Train_Dataset(troot_dir, tcoco_dir, timg_dir, tset_dir, batch_size=8, image_size=352, use_gpu=True)
model.Val_Dataset(vroot_dir, vcoco_dir, vimg_dir, vset_dir)

model.Model(model_name="resnet34"); # resnet 50 brought cuda memory error.
model.Set_Hyperparams(lr=0.0001, val_interval=1, print_interval=20)
model.Train(num_epochs=300,output_model_name="karen_model.pt");

In [ ]:
from infer_detector import Infer
gtf = Infer();
gtf.Model(model_path="/content/karen_model.pt");


In [ ]:
#predictions are quite bad at the moment.
class_list=[]
with open("/content/Root_Dir/Coco_style/annotations/classes.txt") as file:
  for line in file:
    class_list.append(line.rstrip("\n"))
class_list=class_list[:-1]
img_p="/content/Images_and_Labels/Images/0000002_00005_d_0000014_jpg.rf.555bf2106d899e56d45da0a48295f04c.jpg"
scores, labels, boxes = gtf.Predict(img_p, class_list, vis_threshold=0.4);
from IPython.display import Image
Image(filename='output.jpg')